In [2]:
%matplotlib inline

import imageio
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [28]:
from tqdm import tqdm_notebook

## FUNCTIONS

In [3]:
def readVideoFrames(filename):
    vid = imageio.get_reader(filename,  'ffmpeg')
    fps = vid.get_meta_data()['fps']
    num = 0
    frames = []
    while 1:
        try:
            image = vid.get_data(num)
            frames.append(image)
            num+=1
        except IndexError:
            break

    return np.array(frames), fps

In [4]:
def makeGrayScale(rgbVid):
    '''Converts numpy array vid to grayscale vid'''
    frames_bw = []
    for frame in rgbVid:
        frames_bw.append(np.around(np.dot(frame[:,:,:3], [0.2989, 0.587, 0.114])))
    return np.array(frames_bw)

In [5]:
def calculateDifferenceFrames(frames):
    dif_lst = []
    for i in range(1, frames.shape[0]):
        dif_lst.append(frames[i-1,:,:]-frames[i,:,:])
    return np.array(dif_lst)

In [6]:
def divideToMacroblocks(frame, macroblock_size=16):
    macroblocks = []
    m, n = frame.shape
    for i in range(0, m, macroblock_size):
        for j in range(0, n, macroblock_size):
            macroblock = frame[i:i+macroblock_size,j:j+macroblock_size]
    #         print (macroblock.shape)
            if macroblock.shape == (macroblock_size, macroblock_size):
    #             print ('works')
                macroblocks.append(macroblock)
            else:
    #             print ('something\'s goin\' on')
                try:
                    macroblock = np.vstack((macroblock, np.zeros(macroblock.shape[0], macroblock_size-macroblock.shape[1])))
                except TypeError:
                    pass
                try:
                    macroblock = np.hstack((macroblock, np.zeros(macroblock_size-macroblock.shape[0], macroblock.shape[1])))
                except TypeError:
                    pass
                macroblocks.append(macroblock)
    return np.array(macroblocks).reshape((int(m/macroblock_size), int(n/macroblock_size), macroblock_size, macroblock_size))

In [7]:
def createNeighborhood(referenceFrame, indexOfMacroblock, macroblock_size=16, k=16):
    neighborhood = []
#     print (indexOfMacroblock)
    for i in range(indexOfMacroblock[0]-k, indexOfMacroblock[0]+k+1, k):
        for j in range(indexOfMacroblock[1]-k, indexOfMacroblock[1]+k+1, k):
            if (i >= 0 and j >= 0 and i+macroblock_size < referenceFrame.shape[0] and j+macroblock_size < referenceFrame.shape[1]):
#                 print (i,j)
                neighborhood.append(referenceFrame[i:i+macroblock_size, j:j+macroblock_size])
            else:
                neighborhood += [None]
    return neighborhood

In [8]:
def SAD(referenceMacroblock, targetMacroblock):
#     print (targetMacroblock.shape, referenceMacroblock.shape)
    return np.sum(np.abs(targetMacroblock - referenceMacroblock))

In [9]:
def calculateSAD(targetMacroblock, referenceFrame_neighbor_macroblocks):
    SADvals = []
        
    for macroblock in referenceFrame_neighbor_macroblocks:
        if macroblock is not None:
            SADvals.append(SAD(macroblock, targetMacroblock))
        else:
            SADvals.append(np.Inf)
    
    return np.array(SADvals).reshape((3,3))

In [10]:
def logarithmicSearch(referenceFrame, targetMacroblock, indexOfMacroblock, macroblock_size=16, k=16):
    if (k == 0):
        return indexOfMacroblock, referenceFrame[indexOfMacroblock[0]:indexOfMacroblock[0]+macroblock_size, indexOfMacroblock[1]:indexOfMacroblock[1]+macroblock_size] # motionVector END (To_WIDTH, To_HEIGHT), return Predicted Frame
    
    referenceFrame_neighbor_macroblocks = createNeighborhood(referenceFrame, indexOfMacroblock, macroblock_size, k)

    SAD_values = calculateSAD(targetMacroblock, referenceFrame_neighbor_macroblocks)
#     print (SAD_values)
    indexofMinimumSAD = divmod(SAD_values.argmin(), SAD_values.shape[1])
    newIndexOfMacroblock = list(indexOfMacroblock)
    
    if (indexofMinimumSAD[0] == 0):
        newIndexOfMacroblock[0] = indexOfMacroblock[0] - k
    elif (indexofMinimumSAD[0] == 2):
        newIndexOfMacroblock[0] = indexOfMacroblock[0] + k
    
    if (indexofMinimumSAD[1] == 0):
        newIndexOfMacroblock[1] = indexOfMacroblock[1] - k
    elif (indexofMinimumSAD[1] == 2):
        newIndexOfMacroblock[1] = indexOfMacroblock[1] + k

    if (indexofMinimumSAD[0] == 1 and indexofMinimumSAD[1] == 1):
        newK = k//2
    else:
        newK = k       
#     print (indexofMinimumSAD)
#     print (newIndexOfMacroblock)
    return logarithmicSearch(referenceFrame, targetMacroblock, tuple(newIndexOfMacroblock), macroblock_size, newK)

In [11]:
def motionCompensation(referenceFrame, targetFrame, macroblock_size=16):
    predictedBlocks = []
    motionVectors = []
    
    targetMacroblocks = divideToMacroblocks(targetFrame, macroblock_size)
    for i in range(targetMacroblocks.shape[0]):
        for j in range(targetMacroblocks.shape[1]):
            motionVectorSTART = (i*macroblock_size, j*macroblock_size)
            indexofBlock = (i*macroblock_size, j*macroblock_size)
            motionVectorEND, prediction = logarithmicSearch(referenceFrame, targetMacroblocks[i,j,:,:], indexofBlock)
            predictedBlocks.append(prediction)
            motionVectors.append(motionVectorSTART+motionVectorEND)

#     print (len(motionVectors))
    predictedBlocks = np.array(predictedBlocks).reshape(targetMacroblocks.shape)
    motionVectors = np.array(motionVectors, dtype=(int,4)).reshape((targetMacroblocks.shape[0], targetMacroblocks.shape[1], 4))
    return predictedBlocks, motionVectors

In [12]:
def imageReconstructFromBlocks(blocks):
    lines = []
    for i in range(blocks.shape[0]):
        line = []
        for j in range(blocks.shape[1]):
            line.append(blocks[i,j,:,:])
        line = np.hstack(line)
        lines.append(line)
    return np.vstack(lines)

In [14]:
def group_consecutives(vals, step=1):
    """Return list of consecutive lists of numbers from vals (number list)."""
    run = []
    result = [run]
    expect = None
    for v in vals:
        if (v == expect) or (expect is None):
            run.append(v)
        else:
            run = [v]
            result.append(run)
        expect = v + step
    return result

In [40]:
def make_var_list(mbs):
    ''' Make a list (a,x,y)
        a = max value of variance in each macroblock
        x,y = macroblock's coordinates
        
        input: macroblocks of differnce image'''
    lst = []
    for y, macby in enumerate(mbs):
        for x, macbx in enumerate(macby):
            lst.append((max(pd.DataFrame(macbx).var()), y, x))
            
    return lst

In [16]:
def swap_object(coords, frame, back):
    ''' Swap frame's macroblocks given as coords with the same ones from back '''
    tmp = frame
    for tup in coords:
        y,x = tup
        tmp[y][x] = back[y][x]
        
    return tmp

In [17]:
def reconstruct_vid(img1, diffs):
    ''' Reconstruct a video from the difference frames and the 1st frame'''
    final_vid  = [img1]
    cnt = 1
    for frame in diffs:
        final_vid.append(final_vid[-1]-frame)
        
    
    return np.array(final_vid)

In [19]:
def has_n_neighbor(point, lst, num=1):
    ''' Return True if point has atleast #num of neighbors in list lst (other than itself)'''
    neighs = []
    for cand in lst:
        if abs(cand[0]-point[0])<=1 and abs(cand[1]-point[1])<=1 and point != cand:
            neighs.append(cand)
            
    if len(neighs)>=num:
        return True
    else:
        return False
        
        

In [20]:
def return_range(lst, min_neighs=1):
    '''Return the full area that coords of lst cover, creating a rectangle around said coords. 
        Used for clustering macroblocks for better frame swapping'''
    neighs=  []
    final_range = []
    for point in lst:
        if has_n_neighbor(point, lst, num=min_neighs):
            neighs.append(point)
    
    xs, ys = [val[0] for val in neighs], [val[1] for val in neighs]
    
    if len(xs)==0 or len(ys)==0:
        return []
    for i in range(min(xs), max(xs)+1):
        for j in range(min(ys), max(ys)+1):
            final_range.append((i,j))
            
    return final_range

In [21]:
def return_mem(lst):
    ''' Returns the set list that contains the macroblocks to be swap from all the lists inside lst'''
    tmp = []
    for el in lst:
        tmp += el
        
    return list(set(tmp))

In [115]:
def find_moving_object_v2(lst):
    ''' Returns the coordinates of the macroblocks that have a higher max variance value than the upper inner fence'''
    vals = np.array([val[0] for val in lst])
    Q75 = np.percentile(vals, 75)
    iqr = Q75 - np.percentile(vals, 25)
    
    outs = []
    for tup in lst:
        if tup[0]>Q75+(1.5*iqr):
            outs.append((tup[1], tup[2]))
    
    return outs

In [122]:
def save_vid(frames, resources_path='.', name='sample'):
    ''' Saves video '''
    writer = imageio.get_writer(os.path.join(resources_path,name+'.mp4'), fps=fps,mode="I")

    for frame in frames:
        writer.append_data(frame)
    writer.close()

## Main

In [86]:
hall_monitor_cif = "resources/ex617/hall_monitor_cif.y4m"
resources_path = "resources/ex818"

In [87]:
videoFrames, fps = readVideoFrames(hall_monitor_cif)

In [124]:
videoFramesSample = videoFrames[:, :, :, :]
videoFramesSample = makeGrayScale(videoFramesSample)

In [123]:
# diff frames to keep as memory and size of macroblocks
keep_memory = 0
size_of_mbs = 16

# append the 1st sample to the end video and divide it to macroblocks. Keep those as background
finalVideoFramesSample = [videoFramesSample[0]]
mb_back = divideToMacroblocks(videoFramesSample[0],size_of_mbs)

# init memory
mbs = [[] for _ in range(keep_memory)]

# calc difference frames
FrameDifferenceSample = calculateDifferenceFrames(videoFramesSample)

for cnt, frame in enumerate(FrameDifferenceSample):
    # divide the difference frame to macroblocks
    mc = divideToMacroblocks(frame,size_of_mbs)
    # make a list of the max variance between color values per macroblock and the macroblock coordinates. Sort it based on the value
    lst = make_var_list(mc)
    lst.sort(key=lambda x: x[0])
    # find the moving object inside the frame using outlier detection techniques
    moving_mbs = find_moving_object_v2(lst)
    # return the area covered of macroblocks that have atleast 1 neighbor. Cluster those in a sence
    final_mbs = return_range(moving_mbs, min_neighs=1)
    # append the coordinates of the macroblocks that are to be removed to the mem list
    mbs.append(final_mbs)
    # devide the new simple frame to macroblocks
    mb_new = divideToMacroblocks(videoFramesSample[cnt+1],size_of_mbs)
    # return the full set of macroblocks to change , using both the one created from the last frame and n-1 more from the past    
    full_mb = return_mem(mbs[-keep_memory:])
    # create the new frame by swaping said macroblocks with background ones
    new_frame = swap_object(full_mb,mb_new, mb_back)
    # reconstruct and append the image to the final list
    final_img = imageReconstructFromBlocks(new_frame)
    finalVideoFramesSample.append(final_img)

save_vid(finalVideoFramesSample, resources_path, 'hall_monitor_cif_exafanizol')

/anaconda3/lib/python3.6/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [0.0, 255.0]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
